# A: Thu thập dữ liệu cách parse HTML
---

## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
import os
import csv
import pickle
from datetime import datetime, timedelta
import concurrent.futures

---

## Setup ở mức toàn cục

Các setup cần thiết: 
- Cần kiểm tra đã có thư mục Crawl_data hay chưa nếu chưa thì ta cần tạo thư mục mới.
- Khi parse HTML, chúng ta sẽ cho sleep giữa các lần thực hiện request do đó set biến sleep_time = 1.
- Thiết lập cache để giảm số lần thu thập dữ liệu: thời gian hết hạn là None.

In [2]:
if not os.path.exists('Crawl_data/'):
    os.makedirs('Crawl_data/')

In [3]:
sleep_time = 1

In [4]:
requests_cache.install_cache(expire_after=None)

Ngoài ra, em setup những cài đặt cần thiết cho driver selenium cho toàn bộ bài làm này với 1 số thuộc tính cơ bản, trong đó có phần `headless` nghĩa là sẽ chạy selenium dưới dạng ngầm định và giúp giảm thời gian chạy của bài làm

In [5]:
option = Options()

option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")
option.add_argument("--headless")
option.add_experimental_option("excludeSwitches", ['enable-automation']);

# Pass the argument 1 to allow and 2 to block
option.add_experimental_option("prefs", { 
    "profile.default_content_setting_values.notifications": 2,
    "profile.managed_default_content_settings.images": 2
})

---

## Quy trình thực hiện bài làm:
- Bước 1: Thu thập tất cả các thể loại music hiện có trên soundcloud (30 thể loại music)
- Bước 2: Tạo ra các link tìm kiếm từ các thể loại nhạc thu thập được để tìm kiếm các playlist liên quan thuộc thể loại nhạc đó. (30 link tìm kiếm)
- Bước 3: Với mỗi link ở B2, lấy khoảng 50 link `playlist` của thể loại đó và lưu tất cả chúng vào biến `playlist_link` có kiểu dữ liệu là set (tránh trùng lặp playlist)
- Bước 4: Với mỗi playlist_link, lấy ra tên `user` của người tạo ra playlist và lưu vào biến `user_link` có kiểu dữ liệu là set (tránh trùng lặp user)
- Bước 5: Với mỗi playlist_link, lấy ra link của 2 `track` đầu tiên và lưu vào biến `track_link` có kiểu dữ liệu là set (tránh trùng lặp track)

Sau khi hoàn thành bước 5, lúc này chúng ta có 3 biến là `playlist_link`, `user_link` và `user_link` chứa đường link tới các trang cần thu thập dữ liệu.
- Bước 6: Thu thập dữ liệu `playlist` bằng cách truy cập vào các link trong `playlist_link` và lưu vào file
- Bước 7: Thu thập dữ liệu `user` bằng cách truy cập vào các link trong `user_link` và lưu vào file
- Bước 8: Thu thập dữ liệu `track` bằng cách truy cập vào các link trong `track_link` và lưu vào file

## Một số lưu ý khi chạy file:
- Đối với bài làm của em mỗi khi chúng ta `Restart - Run all` thì sẽ cho ra 1 kết quả khác nhau, lí do là vì em làm bài theo hướng universal do đó mỗi khi dùng link để tìm kiếm trên soundcloud thì sẽ thu lại được kết quả khác nhau.
- Những dòng markdown em ghi bên dưới có số liệu kèm theo có thể sẽ khác với lúc thầy chạy bài.

---

## Bước 1: Thu thập tất cả các thể loại music hiện có trên soundcloud (30 thể loại music)

Chúng em đã thu thập được 30 thể loại music hiện có trên soundcloud và ghi chúng vào file `genre_file.txt` và giờ đọc lại file để có list những genre nhạc.

In [6]:
genre_list = []
with open('genre_file.txt', 'r', encoding="utf-8") as fin:
        data = fin.read().strip()
        genre_list = data.split('\n')

In [7]:
genre_list

['Alternative Rock',
 'Ambient',
 'Classical',
 'Country',
 'Dance & EDM',
 'Dancehall',
 'Deep House',
 'Disco',
 'Drum & Bass',
 'Dubstep',
 'Electronic',
 'Folk & Singer-Songwriter',
 'Hip-hop & Rap',
 'House',
 'Indie',
 'Jazz & Blues',
 'Latin',
 'Metal',
 'Piano',
 'Pop',
 'R&B & Soul',
 'Reggae',
 'Reggaeton',
 'Rock',
 'Soundtrack',
 'Techno',
 'Trance',
 'Trap',
 'Triphop',
 'World']

---

## Bước 2: Tạo ra các link tìm kiếm từ các thể loại nhạc thu thập được để tìm kiếm các playlist liên quan thuộc thể loại nhạc đó. (30 link tìm kiếm)

Từ list các genre nhạc, em sẽ tạo ra link tìm kiếm có dạng `f'https://soundcloud.com/search/sets?q={genre}&filter.genre_or_tag={genre}'` dạng f-string để tạo ra các link tìm kiếm. Ở đây `https://soundcloud.com/search/sets?q=` là đường dẫn tiền tố giúp chúng ta tìm được các playlist, `genre` là thể loại nhạc, `&filter.genre_or_tag` là tag giúp chúng ta tìm theo thể chính thể loại nhạc đó. Em chuẩn hóa chuỗi bằng cách thay `" "` thành `"%20"` và `"&"` thành `"%26"` để dể dàng tạo ra link.

In [8]:
search_url = []
for genre in genre_list:
    genre = genre.replace(" ", "%20")
    genre = genre.replace("&", "%26")
    url = f'https://soundcloud.com/search/sets?q={genre}&filter.genre_or_tag={genre}'
    search_url.append(url)

In [9]:
search_url

['https://soundcloud.com/search/sets?q=Alternative%20Rock&filter.genre_or_tag=Alternative%20Rock',
 'https://soundcloud.com/search/sets?q=Ambient&filter.genre_or_tag=Ambient',
 'https://soundcloud.com/search/sets?q=Classical&filter.genre_or_tag=Classical',
 'https://soundcloud.com/search/sets?q=Country&filter.genre_or_tag=Country',
 'https://soundcloud.com/search/sets?q=Dance%20%26%20EDM&filter.genre_or_tag=Dance%20%26%20EDM',
 'https://soundcloud.com/search/sets?q=Dancehall&filter.genre_or_tag=Dancehall',
 'https://soundcloud.com/search/sets?q=Deep%20House&filter.genre_or_tag=Deep%20House',
 'https://soundcloud.com/search/sets?q=Disco&filter.genre_or_tag=Disco',
 'https://soundcloud.com/search/sets?q=Drum%20%26%20Bass&filter.genre_or_tag=Drum%20%26%20Bass',
 'https://soundcloud.com/search/sets?q=Dubstep&filter.genre_or_tag=Dubstep',
 'https://soundcloud.com/search/sets?q=Electronic&filter.genre_or_tag=Electronic',
 'https://soundcloud.com/search/sets?q=Folk%20%26%20Singer-Songwriter&f

---

## Bước 3: Với mỗi link ở B2, lấy khoảng 50 link playlist của thể loại đó và lưu tất cả chúng vào biến playlist_link có kiểu dữ liệu là set (tránh trùng lặp playlist)

Ở bước này, em sẽ dùng selenium để truy cập vào 30 đường link tìm kiếm đã tạo được từ B2, tuy nhiên nếu chỉ đơn thuần gọi 1 driver selenium và chạy nó để truy cập 30 link và thu thập 50 playlist sẽ khá lâu và tốn thời gian, do đó em áp dụng `concurrent.futures` để có thể chạy cùng lúc là 5 driver selenium và mỗi driver sẽ chịu trách nhiệm tìm kiếm cho 6 link giúp giảm bớt thời gian thực thi đoạn code. (Chạy đa luồng)

Đầu tiên, em viết hàm `get_playlist_link`. Hàm này có các input:
- `search_url`: chính là danh sách các link tìm kiếm đã có được ở B2.
- `start`: vị trí bắt đầu của url cần làm việc khi chạy từng driver selenium.

In [10]:
def get_playlist_link(search_url, start):
    
    driver = webdriver.Chrome(options=option, executable_path='chromedriver')
    
    for url in search_url[start:start+6]:
        driver.get(url)
        time.sleep(sleep_time)
        playlist = []
        while len(playlist) < 50:
            playlist = (driver.find_elements(By.CSS_SELECTOR, "li[class='searchList__item']"))
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(1)
        for ele in playlist:
            link = ele.find_element(By.CSS_SELECTOR, "a[class='sound__coverArt']")
            link = link.get_attribute('href')
            playlist_link.add(link)
    
    driver.quit()

- Sử dụng đa luồng để thực hiện

In [11]:
playlist_link = set()
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_playlist_link, search_url, start) for start in range(0, 30, 6)}

In [12]:
# TEST
assert len(playlist_link) > 1000
len(playlist_link)

1473

Như vậy sau khi thực hiện xong, em thu thập được `playlist_link` có độ dài là **1490** playlist, hoàn thành bước **3**.

---

## Bước 4: Với mỗi playlist_link, lấy ra tên user của người tạo ra playlist và lưu vào biến user_link có kiểu dữ liệu là set (tránh trùng lặp user)

In [13]:
def create_user_link_list(playlist_link):
    user_link = set()
    for link in playlist_link:
        flag = link.find("/sets/")
        user_link.add(link[0:flag])
    return user_link

In [14]:
user_link = create_user_link_list(playlist_link)

In [15]:
# TEST
assert len(user_link) > 1000
len(user_link)

1342

Như vậy sau khi thực hiện xong, em thu thập được `user_link` có độ dài là **1354** user, hoàn thành bước **4**.

---

## Bước 5: Với mỗi playlist_link, lấy ra link của 2 track đầu tiên và lưu vào biến track_link có kiểu dữ liệu là set (tránh trùng lặp track)

Tương tự như B3, ở bước này em tiếp tục dùng `concurrent.futures` để chạy đa luồng nhằm tăng tốc độ thu thập dữ liệu.

Đầu tiên, em viết hàm `create_track_link_list`. Hàm này có các input:
- `playlist_link`: chính là danh sách các playlist_link tìm được ở B3.
- `pre_url`: là chuỗi `https://soundcloud.com`
- `start`: vị trí bắt đầu của `playlist_link` khi chạy 1 luồng mới
- `step`: bước nhảy, ở đây step bằng int(len(playlist_link) / 10) vì em muốn chạy 10 luồng

In [16]:
def create_track_link_list(playlist_link, pre_url, start, step):
    
    playlist_link = list(playlist_link)
    for url in playlist_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        tracks = soup.select('article[itemprop=track]')[:2]
        for track in tracks:
            link = track.find("a")["href"]
            track_link.add(pre_url + link)

- Sử dụng đa luồng để thực hiện

In [17]:
track_link = set()
step = int(len(playlist_link) / 10)
pre_url = "https://soundcloud.com"
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(create_track_link_list, playlist_link, pre_url, start, step) for start in range(0, len(playlist_link), step)}

In [18]:
# TEST
assert len(track_link) > 1000
len(track_link)

2825

Như vậy sau khi thực hiện xong, em thu thập được `track_link` có độ dài là **2863** track, hoàn thành bước **5**.

---

Trong các bước 6, 7 và 8, em sẽ tiến hành đi parse dữ liệu bằng cách truy cập vào các link đã lấy được từ các bước 3, 4 và 5. Tuy nhiên, trong quá trình làm bài, em phát hiện ra trong page source của playlist_link, user và track thì sẽ luôn có 1 đoạn script có thể lấy được mà nó chứa toàn bộ tất cả các thông tin mà chúng ta cần thu thập đối với mỗi kiểu đối tượng. Đoạn text này ở dưới dạng dictionary do đó em sẽ get đoạn text này bằng Beautiful Soup và dùng json.loads để chuyển đoạn text này về thành dictionary.

## Bước 6: Thu thập dữ liệu playlist bằng cách truy cập vào các link trong file playlist_link

Hàm `get_info_playlist` có các thành phần:
- `playlist_link`: chứa các link playlist
- `start`, `step`: ví trí bắt đầu và step của một luồng thực hiện

In [19]:
def get_info_playlist(playlist_link, start, step):
    playlist_link = list(playlist_link)
    for url in playlist_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        scripts = soup.find_all("script")
        
        # tìm kiếm script có chứa đoạn text: window.__sc_hydration
        for script in scripts:
            if len(script.contents) != 0:
                if "window.__sc_hydration" in str(script.contents[0]):
                    temp = str(script.contents[0])
                    
        pos = temp.find('{"artwork_url"')
        data = temp[pos:len(temp) - 3]
        data = json.loads(data)
        
        data.pop("user")  # vì trong data còn có key user lưu trữ thông tin của người đăng playlist do đó loại bỏ nó ra
        
        # lấy các track ids có trong playlist và tạo chuỗi theo yêu cầu đề bài
        tracks = data.pop("tracks")
        tracksid = ""
        for track in tracks:
            tracksid += str(track["id"]) + ","
        tracksid = tracksid.rstrip(',')
        data["TracksId"] = tracksid
        playlist_lst.append(data)

- Sử dụng đa luồng để thực hiện

In [20]:
playlist_lst = []
step = int(len(playlist_link) / 10)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_info_playlist, playlist_link, start, step) for start in range(0, len(playlist_link), step)}

### Chuyển đổi dữ liệu thu được về Data frame
- Sau khi thực thi xong đoạn code trên, chúng ta thu được một list các thông tin của các playlist dưới dạng dictionary, để thuận tiện theo dõi cũng như để ghi vào file output, chuyển `playlist_lst` thành 1 Data Frame của `pandas`
- `df_pl` là biến lưu trữ cho `playlist_lst`

In [21]:
df_pl = pd.json_normalize(playlist_lst)
df_pl.columns

Index(['artwork_url', 'created_at', 'description', 'duration', 'embeddable_by',
       'genre', 'id', 'kind', 'label_name', 'last_modified', 'license',
       'likes_count', 'managed_by_feeds', 'permalink', 'permalink_url',
       'public', 'purchase_title', 'purchase_url', 'release_date',
       'reposts_count', 'secret_token', 'sharing', 'tag_list', 'title', 'uri',
       'user_id', 'set_type', 'is_album', 'published_at', 'display_date',
       'track_count', 'url', 'TracksId'],
      dtype='object')

###  Theo dõi 5 dòng đầu của `df_pl`

In [22]:
df_pl.head()

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,url,TracksId
0,https://i1.sndcdn.com/artworks-000222467151-9o...,2017-05-13T21:22:32Z,Thanks for listening and supporting independen...,4438261,all,Electronic,322854746,playlist,None,2017-05-14T09:32:56Z,...,Gentle Dreams ğŸ›�ï¸� - An Indie/Chill/Electro...,https://api.soundcloud.com/playlists/322854746,37994022,,False,2017-05-13T21:22:32Z,2017-05-13T21:22:32Z,18,/alexrainbirdmusic/sets/gentle-dreams-an-indie...,"310826410,323031761,305445147,295247120,287166..."
1,None,2016-04-27T20:48:33Z,None,132180964,all,,219984317,playlist,None,2019-09-14T00:43:01Z,...,TRAP,https://api.soundcloud.com/playlists/219984317,67144491,,False,None,2016-04-27T20:48:33Z,451,/mido-tito-3/sets/trap,"129314155,135305728,246223836,167311586,215001..."
2,https://i1.sndcdn.com/artworks-000145551677-78...,2016-01-21T20:33:40Z,None,103645424,all,Electronic,187981092,playlist,None,2020-12-31T04:39:57Z,...,trap,https://api.soundcloud.com/playlists/187981092,200955727,,False,None,2016-01-21T20:33:40Z,443,/user-548616395/sets/trap,"139912840,191292992,186404225,120687919,128871..."
3,None,2013-12-21T20:36:56Z,None,9434253,none,R&B,17823300,playlist,None,2014-06-13T14:04:07Z,...,TripHop,https://api.soundcloud.com/playlists/17823300,71378711,,False,None,2013-12-21T20:36:56Z,43,/arjay-jean/sets/triphop,"142093,142107,148278,1706674,9748835,23693297,..."
4,https://i1.sndcdn.com/artworks-000366572223-no...,2016-12-29T11:54:48Z,None,774233205,all,Drum & Bass,287371313,playlist,None,2021-10-23T19:45:45Z,...,drum & bass,https://api.soundcloud.com/playlists/287371313,173639213,,False,2016-12-29T11:54:48Z,2016-12-29T11:54:48Z,488,/xj8g5v1h9vp7/sets/dramobass,"299477341,62781091,142721736,178940341,1468531..."


### Ghi dữ liệu ra file `playlist.csv`

In [42]:
playlist_file = "Crawl_data/playlist.csv"
df_pl.to_csv(playlist_file, sep='\t', index=False, encoding="utf-8")

### Test lại dữ liệu đã ghi ra file `playlist.csv`

In [43]:
# TEST
playlists = pd.read_csv(playlist_file, sep='\t')

# Kiểm tra liệu có thu thập đủ trên 1000 records về playlist chưa
assert len(playlists) > 1000

# Xem 5 dòng đầu của dữ liệu
playlists.head()

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,url,TracksId
0,https://i1.sndcdn.com/artworks-000222467151-9o...,2017-05-13T21:22:32Z,Thanks for listening and supporting independen...,4438261,all,Electronic,322854746,playlist,NaN,2017-05-14T09:32:56Z,...,Gentle Dreams ğŸ›�ï¸� - An Indie/Chill/Electro...,https://api.soundcloud.com/playlists/322854746,37994022,NaN,False,2017-05-13T21:22:32Z,2017-05-13T21:22:32Z,18,/alexrainbirdmusic/sets/gentle-dreams-an-indie...,"310826410,323031761,305445147,295247120,287166..."
1,NaN,2016-04-27T20:48:33Z,NaN,132180964,all,NaN,219984317,playlist,NaN,2019-09-14T00:43:01Z,...,TRAP,https://api.soundcloud.com/playlists/219984317,67144491,NaN,False,NaN,2016-04-27T20:48:33Z,451,/mido-tito-3/sets/trap,"129314155,135305728,246223836,167311586,215001..."
2,https://i1.sndcdn.com/artworks-000145551677-78...,2016-01-21T20:33:40Z,NaN,103645424,all,Electronic,187981092,playlist,NaN,2020-12-31T04:39:57Z,...,trap,https://api.soundcloud.com/playlists/187981092,200955727,NaN,False,NaN,2016-01-21T20:33:40Z,443,/user-548616395/sets/trap,"139912840,191292992,186404225,120687919,128871..."
3,NaN,2013-12-21T20:36:56Z,NaN,9434253,none,R&B,17823300,playlist,NaN,2014-06-13T14:04:07Z,...,TripHop,https://api.soundcloud.com/playlists/17823300,71378711,NaN,False,NaN,2013-12-21T20:36:56Z,43,/arjay-jean/sets/triphop,"142093,142107,148278,1706674,9748835,23693297,..."
4,https://i1.sndcdn.com/artworks-000366572223-no...,2016-12-29T11:54:48Z,NaN,774233205,all,Drum & Bass,287371313,playlist,NaN,2021-10-23T19:45:45Z,...,drum & bass,https://api.soundcloud.com/playlists/287371313,173639213,NaN,False,2016-12-29T11:54:48Z,2016-12-29T11:54:48Z,488,/xj8g5v1h9vp7/sets/dramobass,"299477341,62781091,142721736,178940341,1468531..."


---

## Bước 7: Thu thập dữ liệu user bằng cách truy cập vào các link trong file user_link

Hàm `get_info_user` có các thành phần:
- `user_link`: chứa các link user
- `start`, `step`: ví trí bắt đầu và step của một luồng thực hiện

In [25]:
def get_info_user(user_link, start, step):
    user_link = list(user_link)
    for url in user_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        scripts = soup.find_all("script")
        for script in scripts:
            if len(script.contents) != 0:
                if "window.__sc_hydration" in str(script.contents[0]):
                    temp = str(script.contents[0])
        pos = temp.find('{"avatar_url"')
        data = temp[pos:len(temp) - 3]
        data = json.loads(data)
        user_lst.append(data)

- Sử dụng đa luồng để thực hiện

In [26]:
user_lst = []
step = int(len(user_link) / 10)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_info_user, user_link, start, step) for start in range(0, len(user_link), step)}

### Chuyển đổi dữ liệu thu được về Data frame
- Sau khi thực thi xong đoạn code trên, chúng ta thu được một list các thông tin của các user dưới dạng dictionary, để thuận tiện theo dõi cũng như để ghi vào file output, chuyển `user_lst` thành 1 Data Frame của `pandas`
- `df_us` là biến lưu trữ cho `user_lst`

In [27]:
df_us = pd.json_normalize(user_lst)
df_us.columns

Index(['avatar_url', 'city', 'comments_count', 'country_code', 'created_at',
       'creator_subscriptions', 'description', 'followers_count',
       'followings_count', 'first_name', 'full_name', 'groups_count', 'id',
       'kind', 'last_modified', 'last_name', 'likes_count',
       'playlist_likes_count', 'permalink', 'permalink_url', 'playlist_count',
       'reposts_count', 'track_count', 'uri', 'urn', 'username', 'verified',
       'station_urn', 'station_permalink', 'url',
       'creator_subscription.product.id', 'visuals.urn', 'visuals.enabled',
       'visuals.visuals', 'visuals.tracking', 'badges.pro',
       'badges.pro_unlimited', 'badges.verified', 'visuals'],
      dtype='object')

###  Theo dõi 5 dòng đầu của `df_us`

In [28]:
df_us.head()

,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,description,followers_count,followings_count,first_name,...,url,creator_subscription.product.id,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking,badges.pro,badges.pro_unlimited,badges.verified,visuals
0,https://i1.sndcdn.com/avatars-000202454384-nin...,Lisboa,1,None,2010-05-11T12:58:36Z,[{'product': {'id': 'creator-pro-unlimited'}}],Enthusiast.,245,1996,,...,/mehigan-de-vizario,creator-pro-unlimited,soundcloud:users:993251,True,"[{'urn': 'soundcloud:visuals:206685', 'entry_t...",NaN,False,True,False,NaN
1,https://i1.sndcdn.com/avatars-000279257908-3cu...,None,0,None,2016-01-25T06:49:28Z,[{'product': {'id': 'free'}}],None,4,37,,...,/matthew1-280736985,free,soundcloud:users:201650157,True,"[{'urn': 'soundcloud:visuals:18574583', 'entry...",NaN,False,False,False,NaN
2,https://i1.sndcdn.com/avatars-000304829422-gco...,None,3,None,2016-10-30T20:31:17Z,[{'product': {'id': 'free'}}],None,3,7,NyukiZz,...,/nyukizz-nkz,free,NaN,NaN,NaN,NaN,False,False,False,NaN
3,https://i1.sndcdn.com/avatars-000018132952-dmn...,,0,None,2012-07-06T10:50:27Z,[{'product': {'id': 'free'}}],None,35,87,Jay,...,/marioojojo,free,NaN,NaN,NaN,NaN,False,False,False,NaN
4,https://i1.sndcdn.com/avatars-000227227733-6s2...,None,17,None,2011-09-26T03:53:39Z,[{'product': {'id': 'free'}}],None,65,212,Aaron,...,/a-will253,free,soundcloud:users:7654069,True,"[{'urn': 'soundcloud:visuals:10062229', 'entry...",NaN,False,False,False,NaN


### Ghi dữ liệu ra file `user.csv`

In [29]:
user_file = "Crawl_data/user.csv"
df_us.to_csv(user_file, sep='\t', index=False, encoding="utf-8")

### Test lại dữ liệu đã ghi ra file `user.csv`

In [30]:
# TEST
users = pd.read_csv(user_file, sep='\t')

# Kiểm tra liệu có thu thập đủ trên 1000 records về user chưa
assert len(users) > 1000

# Xem 5 dòng đầu của dữ liệu
users.head()

,avatar_url,city,comments_count,country_code,created_at,creator_subscriptions,description,followers_count,followings_count,first_name,...,url,creator_subscription.product.id,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking,badges.pro,badges.pro_unlimited,badges.verified,visuals
0,https://i1.sndcdn.com/avatars-000202454384-nin...,Lisboa,1,NaN,2010-05-11T12:58:36Z,[{'product': {'id': 'creator-pro-unlimited'}}],Enthusiast.,245,1996,NaN,...,/mehigan-de-vizario,creator-pro-unlimited,soundcloud:users:993251,True,"[{'urn': 'soundcloud:visuals:206685', 'entry_t...",NaN,False,True,False,NaN
1,https://i1.sndcdn.com/avatars-000279257908-3cu...,NaN,0,NaN,2016-01-25T06:49:28Z,[{'product': {'id': 'free'}}],NaN,4,37,NaN,...,/matthew1-280736985,free,soundcloud:users:201650157,True,"[{'urn': 'soundcloud:visuals:18574583', 'entry...",NaN,False,False,False,NaN
2,https://i1.sndcdn.com/avatars-000304829422-gco...,NaN,3,NaN,2016-10-30T20:31:17Z,[{'product': {'id': 'free'}}],NaN,3,7,NyukiZz,...,/nyukizz-nkz,free,NaN,NaN,NaN,NaN,False,False,False,NaN
3,https://i1.sndcdn.com/avatars-000018132952-dmn...,NaN,0,NaN,2012-07-06T10:50:27Z,[{'product': {'id': 'free'}}],NaN,35,87,Jay,...,/marioojojo,free,NaN,NaN,NaN,NaN,False,False,False,NaN
4,https://i1.sndcdn.com/avatars-000227227733-6s2...,NaN,17,NaN,2011-09-26T03:53:39Z,[{'product': {'id': 'free'}}],NaN,65,212,Aaron,...,/a-will253,free,soundcloud:users:7654069,True,"[{'urn': 'soundcloud:visuals:10062229', 'entry...",NaN,False,False,False,NaN


---

## Bước 8: Thu thập dữ liệu track bằng cách truy cập vào các link trong file track_link

Hàm `get_info_track` có các thành phần:
- `track_link`: chứa các link track
- `start`, `step`: ví trí bắt đầu và step của một luồng thực hiện

In [31]:
def get_info_track(track_link, start, step):
    track_link = list(track_link)
    for url in track_link[start:start+step]:
        r = requests.get(url)
        if not r.from_cache:
            time.sleep(sleep_time)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "html.parser")
        scripts = soup.find_all("script")
        for script in scripts:
            if len(script.contents) != 0:
                if "window.__sc_hydration" in str(script.contents[0]):
                    temp = str(script.contents[0])
        pos = temp.find('{"artwork_url"')
        data = temp[pos:len(temp) - 3]
        data = json.loads(data)
        data.pop('user')
        track_lst.append(data)

- Sử dụng đa luồng để thực hiện

In [32]:
track_lst = []
step = int(len(track_link) / 10)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(get_info_track, track_link, start, step) for start in range(0, len(track_link), step)}

### Chuyển đổi dữ liệu thu được về Data frame
- Sau khi thực thi xong đoạn code trên, chúng ta thu được một list các thông tin của các track dưới dạng dictionary, để thuận tiện theo dõi cũng như để ghi vào file output, chuyển `track_lst` thành 1 Data Frame của `pandas`
- `df_tr` là biến lưu trữ cho `track_lst`

In [33]:
df_tr = pd.json_normalize(track_lst)
df_tr.columns

Index(['artwork_url', 'caption', 'commentable', 'comment_count', 'created_at',
       'description', 'downloadable', 'download_count', 'duration',
       'full_duration', 'embeddable_by', 'genre', 'has_downloads_left', 'id',
       'kind', 'label_name', 'last_modified', 'license', 'likes_count',
       'permalink', 'permalink_url', 'playback_count', 'public',
       'publisher_metadata', 'purchase_title', 'purchase_url', 'release_date',
       'reposts_count', 'secret_token', 'sharing', 'state', 'streamable',
       'tag_list', 'title', 'track_format', 'uri', 'urn', 'user_id', 'visuals',
       'waveform_url', 'display_date', 'station_urn', 'station_permalink',
       'track_authorization', 'monetization_model', 'policy',
       'media.transcodings', 'publisher_metadata.id', 'publisher_metadata.urn',
       'publisher_metadata.contains_music', 'publisher_metadata.explicit',
       'publisher_metadata.artist', 'publisher_metadata.album_title',
       'publisher_metadata.upc_or_ean', 'pu

###  Theo dõi 5 dòng đầu của `df_tr`

In [34]:
df_tr.head()

,artwork_url,caption,commentable,comment_count,created_at,description,downloadable,download_count,duration,full_duration,...,publisher_metadata.writer_composer,publisher_metadata.release_title,publisher_metadata.publisher,publisher_metadata.c_line,publisher_metadata.c_line_for_display,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking,publisher_metadata.iswc
0,https://i1.sndcdn.com/artworks-000068760774-gz...,None,True,656.0,2014-01-23T08:15:54Z,,False,66463.0,338839,338839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/artworks-000076148269-qy...,None,True,59.0,2014-04-03T12:17:30Z,You can find Sebastian Mullaert aka Minilogue ...,False,0.0,3617749,3617749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://i1.sndcdn.com/artworks-000054578860-i0...,None,True,NaN,2013-08-05T15:01:17Z,A young talent that has seen much success in t...,False,NaN,129056,129056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://i1.sndcdn.com/artworks-luBjdGv0PBty-0-...,None,True,68.0,2021-11-10T02:01:22Z,None,False,0.0,112980,112980,...,"Durk Banks, Thomas Moore, Darontez Mayo",Lion Eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://i1.sndcdn.com/artworks-000079325643-k8...,None,True,83.0,2014-05-13T16:40:24Z,Spotify: sptfy.com/93U\niTunes: itunes.apple.c...,False,0.0,214145,214145,...,Milana Tchebotaryov-Zilnik,Trapped in the Music,Milana Tchebotaryov-Zilnik (self-published),NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Ghi dữ liệu ra file `track.csv`

In [35]:
track_file = "Crawl_data/track.csv"
df_tr.to_csv(track_file, sep='\t', index=False, encoding="utf-8")

### Test lại dữ liệu đã ghi ra file `track.csv`

In [36]:
# TEST
tracks = pd.read_csv(track_file, sep='\t')

# Kiểm tra liệu có thu thập đủ trên 1000 records về track chưa
assert len(tracks) > 1000

# Xem 5 dòng đầu của dữ liệu
tracks.head()

,artwork_url,caption,commentable,comment_count,created_at,description,downloadable,download_count,duration,full_duration,...,publisher_metadata.writer_composer,publisher_metadata.release_title,publisher_metadata.publisher,publisher_metadata.c_line,publisher_metadata.c_line_for_display,visuals.urn,visuals.enabled,visuals.visuals,visuals.tracking,publisher_metadata.iswc
0,https://i1.sndcdn.com/artworks-000068760774-gz...,NaN,True,656.0,2014-01-23T08:15:54Z,NaN,False,66463.0,338839,338839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://i1.sndcdn.com/artworks-000076148269-qy...,NaN,True,59.0,2014-04-03T12:17:30Z,You can find Sebastian Mullaert aka Minilogue ...,False,0.0,3617749,3617749,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://i1.sndcdn.com/artworks-000054578860-i0...,NaN,True,NaN,2013-08-05T15:01:17Z,A young talent that has seen much success in t...,False,NaN,129056,129056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://i1.sndcdn.com/artworks-luBjdGv0PBty-0-...,NaN,True,68.0,2021-11-10T02:01:22Z,NaN,False,0.0,112980,112980,...,"Durk Banks, Thomas Moore, Darontez Mayo",Lion Eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://i1.sndcdn.com/artworks-000079325643-k8...,NaN,True,83.0,2014-05-13T16:40:24Z,Spotify: sptfy.com/93U\niTunes: itunes.apple.c...,False,0.0,214145,214145,...,Milana Tchebotaryov-Zilnik,Trapped in the Music,Milana Tchebotaryov-Zilnik (self-published),NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

## Phân tích và xử lí dữ liệu thu được

### **1) Phân tích đối với tập playlist:**

Sử dụng lại biến `playlists` (Data Frame) đã khai báo ở B6.

In [44]:
print(f'Số dòng và số cột dữ liệu của file playlist.csv {playlists.shape}')
print(f'Số dòng dữ liệu {playlists.shape[0]}')
print(f'Số cột dữ liệu {playlists.shape[1]}')

Số dòng và số cột dữ liệu của file playlist.csv (1473, 33)
Số dòng dữ liệu 1473
Số cột dữ liệu 33


In [45]:
# Xem thông tin 5 dòng đầu tiên
playlists.head()

,artwork_url,created_at,description,duration,embeddable_by,genre,id,kind,label_name,last_modified,...,title,uri,user_id,set_type,is_album,published_at,display_date,track_count,url,TracksId
0,https://i1.sndcdn.com/artworks-000222467151-9o...,2017-05-13T21:22:32Z,Thanks for listening and supporting independen...,4438261,all,Electronic,322854746,playlist,NaN,2017-05-14T09:32:56Z,...,Gentle Dreams ğŸ›�ï¸� - An Indie/Chill/Electro...,https://api.soundcloud.com/playlists/322854746,37994022,NaN,False,2017-05-13T21:22:32Z,2017-05-13T21:22:32Z,18,/alexrainbirdmusic/sets/gentle-dreams-an-indie...,"310826410,323031761,305445147,295247120,287166..."
1,NaN,2016-04-27T20:48:33Z,NaN,132180964,all,NaN,219984317,playlist,NaN,2019-09-14T00:43:01Z,...,TRAP,https://api.soundcloud.com/playlists/219984317,67144491,NaN,False,NaN,2016-04-27T20:48:33Z,451,/mido-tito-3/sets/trap,"129314155,135305728,246223836,167311586,215001..."
2,https://i1.sndcdn.com/artworks-000145551677-78...,2016-01-21T20:33:40Z,NaN,103645424,all,Electronic,187981092,playlist,NaN,2020-12-31T04:39:57Z,...,trap,https://api.soundcloud.com/playlists/187981092,200955727,NaN,False,NaN,2016-01-21T20:33:40Z,443,/user-548616395/sets/trap,"139912840,191292992,186404225,120687919,128871..."
3,NaN,2013-12-21T20:36:56Z,NaN,9434253,none,R&B,17823300,playlist,NaN,2014-06-13T14:04:07Z,...,TripHop,https://api.soundcloud.com/playlists/17823300,71378711,NaN,False,NaN,2013-12-21T20:36:56Z,43,/arjay-jean/sets/triphop,"142093,142107,148278,1706674,9748835,23693297,..."
4,https://i1.sndcdn.com/artworks-000366572223-no...,2016-12-29T11:54:48Z,NaN,774233205,all,Drum & Bass,287371313,playlist,NaN,2021-10-23T19:45:45Z,...,drum & bass,https://api.soundcloud.com/playlists/287371313,173639213,NaN,False,2016-12-29T11:54:48Z,2016-12-29T11:54:48Z,488,/xj8g5v1h9vp7/sets/dramobass,"299477341,62781091,142721736,178940341,1468531..."


#### 1.1) Loại bỏ các cột không thật sự cần thiết
- Có những cột số có số giá trị NaN(rỗng) quá nhiều và không đóng góp nhiều vào dữ liệu, do đó đối với những cột dữ liệu có số dữ liệu rỗng lớn hơn 40% thì em sẽ xóa cột dữ liệu đó đi.
- `del_col_pl` để chứa các cột cần xóa

In [77]:
del_col_pl = []
data = {}

for col in playlists:
    percent = playlists[col].isna().sum() / len(playlists)
    if percent > 0.4:
        data[col] = [percent * 100]
        del_col_pl.append(col)

# hiển thị những cột sẽ xóa đi
if (len(data) > 0):
    print("Những cột sẽ bị xóa đi là: ")
    table = pd.DataFrame(data)
    table.index = ["Phần trăm (%)"]
    display(table.T)
else:
    print("Không có cột nào sẽ bị xóa đi")

# Xóa các cột ra khỏi data
playlists = playlists.drop(del_col_pl, axis=1)

Những cột sẽ bị xóa đi là: 


,Phần trăm (%)
artwork_url,57.094365
description,84.860828
label_name,99.592668
purchase_title,99.660557
purchase_url,99.592668
release_date,95.519348
secret_token,100.000000
set_type,100.000000
published_at,44.602851


- Xem lại thông tin của `playlists` sau khi đã loại bỏ các cột không cần thiết

In [ ]:
print(f'Số dòng và số cột dữ liệu của file playlist.csv {playlists.shape}')
print(f'Số dòng dữ liệu {playlists.shape[0]}')
print(f'Số cột dữ liệu {playlists.shape[1]}')

# Xem thông tin 5 dòng đầu tiên
playlists.head()

#### **1.2) Thông tin về các cột dữ liệu còn lại của `playlists`**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa của dữ liệu
|:-----------:|:----------------:|:---------:|
|**created_at**|Thời gian|Thời gian khởi tạo playlist|
|**duration**|Số nguyên|Tổng thời gian các track nhạc trong playlist|
|**embeddable_by**|Chuỗi|
|**genre**|Chuỗi|Thể loại nhạc của playlist|
|**id**|Kiểu phân loại|Định danh cho playlist|
|**kind**|Chuỗi|Loại user|  
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng của tác giả| 
|**license**|Chuỗi|Giấy phép|
|**likes_count**|Số nguyên|Số lượt yêu thích của playlist|
|**managed_by_feeds**|Luận lý|Playlist có được quản lý bởi Feed|
|**permalink**|Chuỗi|  
|**permalink_url**|Chuỗi|Đường dẫn đến trang Soundcloud |  
|**public**|Luận lý|Playlist có công khai hay không?|   
|**reposts_count**|Số nguyên|Số lượt đăng lại playlist|
|**sharing**|Chuỗi|Chế độ chia sẻ công khai hay riêng tư|
|**tag_list**|Chuỗi|
|**title**|Chuỗi|Tiêu đề playlist|    
|**uri**|Chuỗi|Đường dẫn API của playlist|    
|**user_id**|Kiểu phân loại|Định danh của người sở hữu playlist|
|**is_album**|Luận lý|Playlist có phải album hay không|
|**display_date**|Thời gian|Thời điểm playlist hiển thị|   
|**track_counts**|Số nguyên|Số lượng track trong playlist|
|**url**|Chuỗi|Đường dẫn API của playlist|   
|**TracksId**|Chuỗi|Danh sách id của các track trong playlist được ngăn cách bởi ","|

---

### **2) Phân tích đối với tập user:**

Sử dụng lại biến `users` (Data Frame) đã khai báo ở B7.

In [ ]:
print(f'Số dòng và số cột dữ liệu của file user.csv {users.shape}')
print(f'Số dòng dữ liệu {users.shape[0]}')
print(f'Số cột dữ liệu {users.shape[1]}')

In [ ]:
# Xem thông tin 5 dòng đầu tiên
users.head()

#### 2.1) Loại bỏ các cột không thật sự cần thiết
- Có những cột số có số giá trị NaN(rỗng) quá nhiều và không đóng góp nhiều vào dữ liệu, do đó đối với những cột dữ liệu có số dữ liệu rỗng lớn hơn 40% thì em sẽ xóa cột dữ liệu đó đi.
- `del_col_us` để chứa các cột cần xóa

In [78]:
del_col_us = []
data = {}

for col in users:
    percent = users[col].isna().sum() / len(users)
    if percent > 0.4:
        data[col] = [percent * 100]
        del_col_us.append(col)

# hiển thị những cột sẽ xóa đi
if (len(data) > 0):
    print("Những cột sẽ bị xóa đi là: ")
    table = pd.DataFrame(data)
    table.index = ["Phần trăm (%)"]
    display(table.T)
else:
    print("Không có cột nào sẽ bị xóa đi")

# Xóa các cột ra khỏi data
users = users.drop(del_col_us, axis=1)

Những cột sẽ bị xóa đi là: 


,Phần trăm (%)
city,50.670641
country_code,49.329359
description,61.475410
last_name,41.356185
reposts_count,100.000000
visuals.tracking,100.000000
visuals,100.000000


- Xem lại thông tin của `users` sau khi đã loại bỏ các cột không cần thiết

In [ ]:
print(f'Số dòng và số cột dữ liệu của file user.csv {users.shape}')
print(f'Số dòng dữ liệu {users.shape[0]}')
print(f'Số cột dữ liệu {users.shape[1]}')

# Xem thông tin 5 dòng đầu tiên
users.head()

#### **2.2) Thông tin về các cột dữ liệu còn lại của `users`**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa  
|:--------:|:-------------:|:-----------:
|**avatar_url**|Chuỗi |Đường dẫn đến ản đại diện của user|
|**created_at**|Thời gian|Thời gian khởi tạo user|
|**id**|Kiểu phân loại|Định danh của user|
|**comments_count**|Số nguyên|Số lượt bình luận của user|
|**likes_count**|Số nguyên|Số lượt được yêu thích của user|
|**followers_count**|Số nguyên|Số lượt theo dõi của user|
|**followings_count**|Số nguyên|Số lượt user theo dõi|
|**playlist_likes_count**|Số nguyên|Số lượt được yêu thích các playlist của user|
|**playlist_count**|Số nguyên|Số playlist của user|
|**track_count**|Số nguyên|Số lượng track của user|
|**kind**|Chuỗi|Loại đối tượng|
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng| 
|**first_name**|Chuỗi|Tên của user|  
|**full_name**|Chuỗi|Tên đầy đủ của user| 
|**username**|Chuỗi|Tên đăng nhập của user|   
|**groups_count**|Số nguyên|Số nhóm của user|
|**permalink**|Chuỗi|Đường dẫn cố định của tài nguyên| 
|**permalink_url**|Chuỗi|Đường dẫn đến trang Soundcloud |   
|**uri**|Chuỗi|Đường dẫn API của user|    
|**url**|Chuỗi|Đường dẫn API của user|    
|**urn**|Chuỗi|Đường dẫn API của user|    
|**creator_subscriptions**|Chuỗi|Loại thành viên của user|    
|**creator_subscription.product.id**|Chuỗi|Loại thành viên của user|    
|**badges.pro_unlimited'**|Luận lý|User có phải người dùng pro_unlimited không|
|**verified**|Luận lý|Xác thực người dùng|
|**visuals.urn**|Chuỗi
|**visuals.enabled**|Chuỗi|
|**visuals.visuals**|Chuỗi|

---

### **3) Phân tích đối với tập track:**

Sử dụng lại biến `tracks` (Data Frame) đã khai báo ở B8.

In [ ]:
print(f'Số dòng và số cột dữ liệu của file track.csv {tracks.shape}')
print(f'Số dòng dữ liệu {tracks.shape[0]}')
print(f'Số cột dữ liệu {tracks.shape[1]}')

In [ ]:
# Xem thông tin 5 dòng đầu tiên
tracks.head()

#### 3.1) Loại bỏ các cột không thật sự cần thiết
- Có những cột số có số giá trị NaN(rỗng) quá nhiều và không đóng góp nhiều vào dữ liệu, do đó đối với những cột dữ liệu có số dữ liệu rỗng lớn hơn 40% thì em sẽ xóa cột dữ liệu đó đi.
- `del_col_tr` để chứa các cột cần xóa

In [80]:
del_col_tr = []
data = {}

for col in tracks:
    percent = tracks[col].isna().sum() / len(tracks)
    if percent > 0.4:
        data[col] = [percent * 100]
        del_col_tr.append(col)

# hiển thị những cột sẽ xóa đi
if (len(data) > 0):
    print("Những cột sẽ bị xóa đi là: ")
    table = pd.DataFrame(data)
    table.index = ["Phần trăm (%)"]
    display(table.T)
else:
    print("Không có cột nào sẽ bị xóa đi")

# Xóa các cột ra khỏi data
tracks = tracks.drop(del_col_tr, axis=1)

Những cột sẽ bị xóa đi là: 


,Phần trăm (%)
caption,99.646018
label_name,73.840708
publisher_metadata,100.000000
purchase_title,76.530973
purchase_url,61.132743
release_date,69.557522
secret_token,100.000000
visuals,100.000000
publisher_metadata.id,40.283186
publisher_metadata.urn,40.283186


- Xem lại thông tin của `tracks` sau khi đã loại bỏ các cột không cần thiết

In [ ]:
print(f'Số dòng và số cột dữ liệu của file user.csv {tracks.shape}')
print(f'Số dòng dữ liệu {tracks.shape[0]}')
print(f'Số cột dữ liệu {tracks.shape[1]}')

# Xem thông tin 5 dòng đầu tiên
tracks.head()

#### **3.2) Thông tin về các cột dữ liệu còn lại của `tracks`**

|Thuộc tính|Kiểu dữ liệu|     Ý nghĩa của dữ liệu
|:-----------:|:----------------:|:---------:|
|**artwork_url**|chuỗi |URL ảnh của track|
|**comment_count**|Số nguyên |Số bình luận của một track|
|**created_at**|Thời gian |Thời gian khởi tạo của track|
|**description**|Chuỗi |Mô tả của track đó do user đăng tải|
|**downloadable**|Luận lý |Track có tải về được không?|
|**download_count**|Số nguyên |Số lượt tải về của track|
|**duration**|Số nguyên |Thời gian phát của track|
|**full_duration**|Số nguyên|Thời gian phát của track|
|**embeddable_by**|Chuỗi|
|**genre**|Chuỗi|Thể loại của track|
|**has_downloads_left**|Luận lý|
|**id**|Kiểu phân loại|Định danh cho track|
|**kind**|Chuỗi|Loại track
|**last_modified**|Thời gian|Lần chỉnh sửa cuối cùng của track|
|**license likes_count**|Số nguyên|Số lượt like có bản giấy phép của track|
|**public**|Luận lý|Có công khai trên soundcloud hay không|
|**permalink**|Chuỗi|
|**permalink_url**|Chuỗi|Đường dẫn dến trang soundcloud|
|**playback_count**|Số nguyên|Số lượt nghe của track|
|**public**|Luận lý|Track có được công khai hay không?|
|**reposts_count**|Số nguyên|Số lượt đăng lại của track|
|**sharing**|Chuỗi|Share đường dẫn của track|
|**state**|Chuỗi|Nút trạng thái share đường dẫn của track|
|**streamable**|Luận lý|Track có được trực tuyến hay không?|
|**tag_list**|Chuỗi|Các danh sách track được gợi tag|
|**title**|Chuỗi|Tiêu đề của track|
|**track_format**|Chuỗi|Định dạng của track|
|**uri**|Chuỗi|Đường dẫn API của track
|**urn**|Chuỗi|Đường dẫn API của track
|**user_id**|Chuỗi|Người chủ của track
|**waveform_url**|Chuỗi|
|**display_date**|Thời gian|Thời gian đăng tải track trên track page của user|
|**station_urn**|Chuỗi|
|**station_permalink**|Chuỗi|
|**track_authorization**|Chuỗi|Bản quyền của track|
|**monetization_model**|Chuỗi|
|**policy**|Chuỗi|Điều khoản, chính sách của track|
|**media.transcodings**|Chuỗi|